In [ ]:
#!/usr/bin/python
import json,os

class pinyin:
    # basic data #
    shengmu = ('b','p','m','f','d','t','n','l','g','k','h','j','q','x','r','w','y','z','c','s','zh','ch','sh')
    yunmu = ('ang','eng','ing','ong','an','en','in','un','ai','ei','ao','ou','iu','er','en','a','o','e','i','u', 'v')
    shengdiao = {'a':['ā','á','ǎ','à'],'o':['ō','ó','ǒ','ò'],'e':['ē','é','ě','è'],'i':['ī','í','ǐ','ì'],'u':['ū','ú','ǔ','ù'],'v':['ǖ','ǘ','ǚ','ǜ']}
    # 当j、q、x、y与ü相拼时，要去掉两点。
    shengdiao_jqxy = {'a':['ā','á','ǎ','à'],'o':['ō','ó','ǒ','ò'],'e':['ē','é','ě','è'],'i':['ī','í','ǐ','ì'],'v':['ū','ú','ǔ','ù']}
    shengdiao_inv = {}
    shengdiao_jqxy_inv = {}
    
    def __init__(self):
        for k in self.shengdiao:
            v = self.shengdiao[k]
            for i in range(4):
                self.shengdiao_inv[v[i]] = [k,i+1]
        for k in self.shengdiao_jqxy:
            v = self.shengdiao_jqxy[k]
            for i in range(4):
                self.shengdiao_jqxy_inv[v[i]] = [k,i+1]
py = pinyin()
print(py.shengdiao_inv)



# load data #
data_folder = os.path.join('..','data')
data_fn = os.path.join(data_folder,'idiom.json')
data_fn_out = os.path.join('idiom_simp.json')

with open(data_fn) as fo:
    idiom_data = json.loads(fo.read())

In [ ]:
def simplify(data):
    idiom_data_simp = [{'word':item['word'],'pinyin':item['pinyin'].split()} for item in data]
    def detach(data_simp):
        for chengyu in data_simp:
            shengmu = []
            yunmu = []
            shengdiao = ''
            for item in chengyu['pinyin']:
                # shengmu
                if item[0:2] in py.shengmu:
                    sm = item[0:2]
                    shengmu.append(sm)
                    ym = item[2:]
                elif item[0:1] in py.shengmu:
                    sm = item[0:1]
                    shengmu.append(sm)
                    ym = item[1:]
                else:
                    sm = ''
                    shengmu.append(sm)
                    ym = item
                # yunmu
                for i in range(len(ym)):
                    ymi0 = ym[i]
                    shengdiao_inv = py.shengdiao_jqxy_inv if sm in 'jqxy' else py.shengdiao_inv
                    if ymi0 in shengdiao_inv:
                        ymi,sd = shengdiao_inv[ymi0]
                        yunmu.append(ym.replace(ymi0,ymi))
                        shengdiao += str(sd)
                        break
                    if i == len(ym) - 1:
                        yunmu.append(ym)
                        shengdiao += '5'
            chengyu['shengmu'] = shengmu
            chengyu['yunmu'] = yunmu
            chengyu['shengdiao'] = shengdiao
            if '55' in shengdiao: print(chengyu)
            yield chengyu
    return list(detach(idiom_data_simp))



# test sample #
num = len(idiom_data)
sample = idiom_data[1::num//10]
num_sample = len(sample)
print(num,num_sample)
print(sample[2])
print(simplify(sample))

In [ ]:
with open(data_fn_out,'w') as fo:
    fo.write(json.dumps(simplify(idiom_data)))
print('done')